In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time
from selenium import webdriver

In [3]:
years = list(range(1990, 2024))

In [4]:
driver = webdriver.Chrome()

In [5]:
url_start = "https://www.basketball-reference.com/leagues/NBA_{}_totals.html"

In [10]:
#Iterate through the years to get the data for all players in each year
for year in years:
    url = url_start.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    
    time.sleep(7)
    html = driver.page_source
    
    with open("seasonTotals/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(html)

In [70]:
# Parse data with BeautifulSoup, delete unwanted sections, and store it into a list to be moved to a csv
data = []
for year in years:
    with open("seasonTotals/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")    
    total_stats_table = soup.find(id="totals_stats")
    soup.find('tr', class_="thead").decompose()
    
    total_stats= pd.read_html(StringIO(str(total_stats_table)))[0]
    total_stats['Year'] = year
    
    data.append(total_stats)

In [72]:
total_stats_agg = pd.concat(data)

In [74]:
total_stats_agg.to_csv("total_stats.csv")